In [179]:
%load_ext autoreload
%autoreload 2
import sys
sys.path.append('../codes')
from torch.utils.data import Dataset, DataLoader
import pickle
import numpy as np
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder
import pandas as pd
import config
import pickle
import config
import networks
import utils
import loss
import trainer
import evaluator
import plots
import torch
from torch  import nn
from scipy.stats import pearsonr
from captum.attr import DeepLift, IntegratedGradients

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [180]:
from sklearn.ensemble import RandomForestRegressor
from scipy.stats import spearmanr, pearsonr

In [181]:
def get_res_by_location(test_data):
    locations = test_data['Loc_no'].unique()
    trials = test_data['trial'].unique()

    result_dict ={}
    result_dict['location'] = []
    result_dict['trial'] = []
    result_dict['num_geno'] = []
    result_dict['pcc'] = []

    for location in locations:
        for trial in trials:
            partial_test = test_data[(test_data['Loc_no'] == location) & (test_data['trial'] == trial)]

            if len(partial_test) > 20:
               
                pcc = pearsonr(partial_test['Value'].to_numpy().reshape(-1,), partial_test['predicted'].to_numpy().reshape(-1,))[0]
                result_dict['location'].append(location)
                result_dict['trial'].append(trial)
                result_dict['num_geno'].append(len(partial_test))
                result_dict['pcc'].append(pcc)
   
    result_df = pd.DataFrame(result_dict)
   
    return result_df

In [182]:
def get_data(file):
    with open(file, 'rb') as pfile:
        all_data = pickle.load(pfile)

    all_data = all_data[all_data['Value'] < 10]
    
    weather = all_data['weather'].tolist()
    weather = np.array(weather)
    
    genotypes = all_data.iloc[:, 3:-14].to_numpy()
    
    target = all_data['Value'].to_numpy()
    
    return genotypes, weather, target

## weather representation

In [183]:
test_loader = utils.create_dataloader_by_env_avg(config.test_unique_env_data, 
                                                        is_training=False)

In [184]:
# test_genotypes, test_weather, test_target = get_data(config.test_data)
test_loader.dataset.data.shape

(133, 81)

In [185]:
mdl_avg_over_geno = networks.fc_avg_net_over_geno(num_features = test_loader.dataset.data.shape[1], hidden_dim=54)
mdl_avg_over_geno.load_state_dict(torch.load(config.model_avg_by_env_path))
criterion = nn.MSELoss()

In [186]:
test_loss, test_true, test_predicted = evaluator.evaluate_step_by_avg(test_loader, mdl_avg_over_geno, criterion)

In [187]:
print('PCC: ', pearsonr(test_true, test_predicted)[0])

PCC:  0.5781606262013186


## Environment importance

In [188]:
mdl_avg_over_geno = networks.fc_avg_net_over_geno(num_features = test_loader.dataset.data.shape[1], hidden_dim=54)
mdl_avg_over_geno.load_state_dict(torch.load(config.model_avg_by_env_path))

<All keys matched successfully>

In [189]:
# # device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
# # device ='cpu'

# # mdl_avg_over_geno.to(device)
# mdl_avg_over_geno.eval()


# # data = test_loader.dataset.data
# # baseline = torch.zeros(data.shape)

# # data = torch.from_numpy(data)
# # data = data.float()
# # data = data.to(device)
# data = torch.rand(50,81)
# # data = data.to(device)

# baseline = torch.zeros(data.shape)

# # baseline = baseline.to(device)


# dl = DeepLift(mdl_avg_over_geno) 
# attributions, delta = dl.attribute(data, baseline, target=0, return_convergence_delta=True)

In [190]:
# data.shape


## genotype representation

In [191]:
test_loader = utils.create_dataloader_by_geno_avg(config.test_unique_env_data, 
                                                        is_training=False)

In [192]:
mdl_avg_over_env = networks.fc_avg_net(num_features = test_loader.dataset.data.shape[1], hidden_dim=2000)
mdl_avg_over_env.load_state_dict(torch.load(config.model_avg_by_geno_path))
criterion = nn.MSELoss()

In [193]:
test_loss, test_true, test_predicted = evaluator.evaluate_step_by_avg(test_loader, mdl_avg_over_env, criterion)

In [194]:
print('PCC: ', pearsonr(test_true, test_predicted)[0])

PCC:  0.29874782167211883
